In the terminal: `cd ..` to be in the `_shared_utils` sub-folder, run `python setup.py install`, then `cd` back into your task sub-folder (e.g., `bus_service_increase`)

In [1]:
import branca
import geopandas as gpd
import pandas as pd

import shared_utils

In [2]:
shared_utils.geography_utils.CA_NAD83Albers

'EPSG:3310'

In [3]:
shared_utils.utils.geoparquet_gcs_export?

Signature: shared_utils.utils.geoparquet_gcs_export(gdf, GCS_FILE_PATH, name)
Docstring:
Save geodataframe as parquet locally, 
then move to GCS bucket and delete local file.

gdf: geopandas.GeoDataFrame
GCS_FILE_PATH: str. Ex: gs://calitp-analytics-data/data-analyses/my-folder/
name: str. Filename.
File:      /opt/conda/lib/python3.9/site-packages/shared_utils-0.1.0-py3.9.egg/shared_utils/utils.py
Type:      function


In [4]:
sample_data = {
    'District' : [1, 2, 3, 1],
    'Population' : [3_000, 2_000, 1_500, 3_000],
    'id' : ['A', 'B', 'C', 'D']
}

df = pd.DataFrame(sample_data)
df

,District,Population,id
0,1,3000,A
1,2,2000,B
2,3,1500,C
3,1,3000,D


In [5]:
group_cols = ["District"]
sum_cols = []
mean_cols = ["Population"]
count_cols = []
nunique_cols = ["id"]

shared_utils.geography_utils.aggregate_by_geography(df, group_cols=group_cols,
                                                    sum_cols = sum_cols,
                                                    mean_cols = mean_cols,
                                                    count_cols = count_cols,
                                                    nunique_cols = nunique_cols)

,District,Population,id
0,1,3000,2
1,2,2000,1
2,3,1500,1


In [6]:
gdf = gpd.read_parquet((
    "gs://calitp-analytics-data/data-analyses/bus_service_increase/"
    "bus_stop_times_by_tract.parquet"
))
gdf = gdf[gdf.County=="Los Angeles"]

gdf.head()

,Tract,ZIP,Population,sq_mi,pop_sq_mi,overall_ptile,pollution_ptile,popchar_ptile,equity_group,pollution_group,...,County,City,geometry,num_jobs,jobs_sq_mi,num_pop_jobs,popjobs_sq_mi,num_arrivals,stop_id,itp_id
1166,06037101110,91042,4283,0.441237,9706.811683,46.280888,29.072806,55.685830,2.0,1,...,Los Angeles,Los Angeles,"POLYGON ((-118.29434 34.26320, -118.29419 34.2...",594,1346.216703,4877,11053.028386,97.0,3.0,2.0
1167,06037101122,91042,3405,1.023417,3327.088245,20.839637,19.502178,25.441251,1.0,1,...,Los Angeles,Los Angeles,"POLYGON ((-118.28527 34.26643, -118.28167 34.2...",236,230.599949,3641,3557.688194,NaN,NaN,NaN
1168,06037101210,91042,6347,0.250344,25353.122339,51.134644,28.400747,64.624307,2.0,1,...,Los Angeles,Los Angeles,"POLYGON ((-118.28592 34.25589, -118.28592 34.2...",1021,4078.389461,7368,29431.511800,232.0,6.0,2.0
1169,06037101220,91042,3702,0.270133,13704.356329,59.947050,36.652147,71.482602,2.0,2,...,Los Angeles,Los Angeles,"POLYGON ((-118.28099 34.25592, -118.28100 34.2...",494,1828.728262,4196,15533.084591,194.0,6.0,2.0
1170,06037101300,91042,3884,0.994814,3904.249217,37.153303,55.731176,27.609682,2.0,2,...,Los Angeles,Los Angeles,"POLYGON ((-118.26538 34.25354, -118.26530 34.2...",958,962.994529,4842,4867.243746,337.0,9.0,2.0


In [7]:
popup_dict = {
    "Tract": "Census Tract",
    "ZIP": "Zip Code",
    "Population": "Pop"
}

colorscale = branca.colormap.StepColormap(
                colors=["gray", "green", "navy"], 
                index=[2_000, 4_000, 8_000],
                vmin=0, vmax=15_000,
)

m = shared_utils.map_utils.make_folium_choropleth_map(gdf,
                                                  plot_col = "Population",
                                                  popup_dict = popup_dict,
                                                  tooltip_dict = popup_dict,
                                                  colorscale = colorscale,
                                                  fig_width = 500, fig_height = 800,
                                                  zoom = 9,
                                                  centroid = shared_utils.map_utils.REGION_CENTROIDS["Los Angeles"][0],
                                                  title="Population in LA"
)

#m